In [3]:
# Import des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [4]:
# Définition des chemins pour les répertoires d'images
train_dir = r"C:\Users\Souad.ABOUD\Documents\LAB\ProjetDL\Dataset_BUSI_with_GT\train"
test_dir = r"C:\Users\Souad.ABOUD\Documents\LAB\ProjetDL\Dataset_BUSI_with_GT\test"


In [5]:
# Normalisation et augmentation des images
datagen = ImageDataGenerator(
    rescale=1./255,          # Normalisation des pixels
    rotation_range=20,       # Rotation aléatoire des images
    zoom_range=0.2,          # Zoom aléatoire
    horizontal_flip=True,    # Flip horizontal
    validation_split=0.2     # 20% des données utilisées pour la validation
)


In [6]:
# Chargement des données d'entraînement
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Taille des images après redimensionnement
    batch_size=32,           # Taille des lots
    class_mode='binary',     # Classification binaire (cancer / normal)
    subset='training'        # 80% des données pour l'entraînement
)

# Chargement des données de validation
val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'      # 20% des données pour la validation
)

print("Prétraitement terminé ✅")


Found 1263 images belonging to 3 classes.
Found 315 images belonging to 3 classes.
Prétraitement terminé ✅


In [7]:
# Définition du modèle CNN
model = models.Sequential([
    layers.InputLayer(input_shape=(224, 224, 3)),  # Utilisation de InputLayer pour la première couche
    layers.Conv2D(32, (3, 3), activation='relu'),   # Première couche Conv2D
    layers.MaxPooling2D(2, 2),                      # MaxPooling pour réduire la taille
    layers.Conv2D(64, (3, 3), activation='relu'),   # Deuxième couche Conv2D
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),  # Troisième couche Conv2D
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),                               # Aplatir les images pour les couches denses
    layers.Dense(128, activation='relu'),           # Couche dense
    layers.Dense(1, activation='sigmoid')           # Sortie binaire (cancer ou normal)
])

# Affichage du résumé du modèle
model.summary()


c:\Users\Souad.ABOUD\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Compilation du modèle
model.compile(optimizer='adam',               # Optimiseur Adam
              loss='binary_crossentropy',     # Fonction de perte pour la classification binaire
              metrics=['accuracy'])          # Utilisation de l'accuracy comme métrique


In [10]:
# Entraînement du modèle
history = model.fit(
    train_data,                # Données d'entraînement
    validation_data=val_data,  # Données de validation
    epochs=10                  # Nombre d'époques
)


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3390 - loss: 0.8070

c:\Users\Souad.ABOUD\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.3398 - loss: 0.8050 - val_accuracy: 0.3651 - val_loss: 0.6594
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.4071 - loss: 0.5880 - val_accuracy: 0.2794 - val_loss: 0.5991
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.4156 - loss: 0.2076 - val_accuracy: 0.3111 - val_loss: -0.2797
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.4605 - loss: -8.5629 - val_accuracy: 0.4063 - val_loss: -90.6822
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.4475 - loss: -787.7599 - val_accuracy: 0.3556 - val_loss: -6938.2402
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.3986 - loss: -16815.4004 - val_accuracy: 0.3619 - val_loss: -69998.3594
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.3967 - loss: -127240.5625 - val_accuracy: 0.5079 - val_loss: -373491.1875
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 39s 971ms/step - accuracy: 0.3779 - loss: -715997.8125 - v

In [11]:
# Sauvegarde du modèle
os.makedirs("model", exist_ok=True)  # Créer le dossier si nécessaire
model.save("model/cancer_detector.h5")  # Sauvegarder le modèle dans un fichier .h5
print("Modèle entraîné et sauvegardé ✅")


Modèle entraîné et sauvegardé ✅
